<a href="https://colab.research.google.com/github/YS-JEOUNG/amazon-review-project/blob/main/code/artscrafts_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split
import gzip
import dask as dd

In [ ]:
df = pd.read_csv('C:/Users/STU/Desktop/arts_total.csv')

In [ ]:
df = df.drop(['vote', 'verified', 'style', 'reviewer_name', 'unix_review_time', 'category',
              'description', 'brand', 'rank', 'main_cat', 'date', 'price', 'review_text',
              'rating_class', 'time'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130133 entries, 0 to 130132
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   overall      130133 non-null  float64
 1   reviewer_id  130133 non-null  object 
 2   asin         130133 non-null  object 
 3   title        130133 non-null  object 
dtypes: float64(1), object(3)
memory usage: 4.0+ MB


In [ ]:
counts_reviewer=df['reviewer_id'].value_counts()
counts_product=df['asin'].value_counts()

In [ ]:
df1 = df[df['asin'].isin(counts_product[counts_product>=50].index)]
df1.shape

(38597, 4)

In [ ]:
ratings_sum = pd.DataFrame(df1.groupby(['asin'])['overall'].sum()).rename(columns = {'overall': 'ratings_sum'})
top10 = ratings_sum.sort_values('ratings_sum', ascending=False).head(10)
top10

,ratings_sum
asin,
B003AVMZA4,2956.0
B004RDH7Y8,2671.0
B005HR1JMO,2598.0
B000F7DPEQ,2283.0
B01D3MWPU4,1483.0
B00WW8KIRI,1411.0
B009CCVS4E,1382.0
B005GXPOAM,1338.0
B000980L02,1234.0


In [ ]:
top10_popular = top10.merge(df1, left_index=True, right_on='asin').drop_duplicates(
    ['asin', 'title'])[['asin', 'title', 'ratings_sum']]

In [ ]:
ratingsd = pd.pivot_table(data=df1, index='asin', columns='reviewer_id', values='overall').fillna(0)
ratingsd.head()

reviewer_id,A0193442L7IJXD7HANDM,A02049035W9C30B2NBJM,A0228801ER3IBSLNOT5W,A0345137PWD0TJ09QK1A,A0429690KG9J3JO6LSSH,A04454666ZIS4FUX7ZME,A0479295RXACIYIXRLRF,A0483767C53I9DYB5TMS,A0490453AYGB1BAAOG64,A0516005NLXNKCMT2YI9,...,AZYKTPOHMZX7N,AZYML1QLCY4U8,AZYP4FQ2L2C4O,AZYQ60ZR354HC,AZYW27SS60XQY,AZZ1CC0BCXZ9H,AZZ74XTNQCOS4,AZZXNFW30OVPU,AZZXW4PH4OEEK,AZZZPAB0MBZ69
asin,,,,,,,,,,,,,,,,,,,,,
B0000302YN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004THXH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004Z2UB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004Z2UD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00006B8FS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ratingsd.shape

(398, 34594)

In [ ]:
traind, testd = train_test_split(ratingsd, test_size=0.3, random_state=42)

In [ ]:
train = traind.values
test = testd.values

In [ ]:
sparsity = float(len(train.nonzero()[0]))
sparsity /= (train.shape[0]*train.shape[1])
sparsity *= 100
print(f'sparsity: {sparsity}%')

sparsity: 0.27993792744032214%


In [ ]:
def item_similarity(ratings, epsilon=1e-3):
    sim = ratings.T.dot(ratings)+epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim/norms/norms.T)

In [ ]:
item_sim = item_similarity(train)

In [ ]:
def predict_item(ratings, similarity):
    return ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

In [ ]:
item_prediction = predict_item(train, item_sim)

In [ ]:
item_prediction[:4, :4]

array([[3.91055208e-04, 3.91055208e-04, 2.11751229e-04, 1.99216687e-04],
       [2.48257030e-04, 2.48257030e-04, 1.34427902e-04, 1.26470488e-04],
       [1.79309185e-04, 1.79309185e-04, 9.70935546e-05, 9.13461349e-05],
       [2.55752835e-04, 2.55752835e-04, 1.38486781e-04, 1.30289103e-04]])

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
def get_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [ ]:
print('item-based CF MSE:', str(get_mse(item_prediction, test)))

item-based CF MSE: 17.369296977860003


In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
r_mat_tr = svd.fit_transform(traind)
print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())

[0.02992958 0.02614082 0.02482213 0.02280331 0.0139896  0.01313408
 0.01243283 0.01114403 0.01082745 0.01086482 0.0102264  0.00967991
 0.00949537 0.00947888 0.00944445 0.00931534 0.00817544 0.0078972
 0.00777677 0.00772837 0.00725354 0.00720834 0.00690552 0.00686204
 0.00680672 0.00603403 0.00626394 0.00625671 0.00606014 0.00598057
 0.00594802 0.00580893 0.0057666  0.00560095 0.00539902 0.00534443
 0.00523616 0.00518476 0.00506533 0.00483486 0.00478473 0.0047079
 0.0045246  0.00454408 0.00452948 0.00449724 0.00447649 0.0043645
 0.00431131 0.00423728]
0.43010498130926217


In [ ]:
ctrain = cosine_similarity(r_mat_tr)

In [ ]:
r_mat_tr = svd.fit_transform(testd)
print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())

[0.03097939 0.03041311 0.0281574  0.02734198 0.02598669 0.02299087
 0.0227991  0.0213169  0.02104761 0.02006568 0.01748098 0.01642129
 0.01569749 0.01584233 0.01439854 0.01396505 0.01331836 0.01313984
 0.01297719 0.01281752 0.01237055 0.01216191 0.01181328 0.01135496
 0.01011608 0.00988023 0.00970572 0.00970603 0.00953923 0.00929233
 0.00936193 0.00900899 0.00868604 0.00858888 0.00856311 0.00817107
 0.00810764 0.00782634 0.00782006 0.00755731 0.00732157 0.00707385
 0.00716187 0.00704263 0.00694895 0.00672118 0.00686985 0.0067374
 0.00640722 0.00652627]
0.6555998450976376


In [ ]:
ctest = cosine_similarity(r_mat_tr)

In [ ]:
print('CF MSE:', str(get_mse(ctrain, ctest)))

CF MSE: 0.07154861573382829


In [ ]:
df1 = df1.sort_values(by='overall')
df1 = df1.reset_index(drop=True)
count_users = df1.groupby('reviewer_id', as_index=False).count()

In [ ]:
count = df1.groupby('asin', as_index=False).mean()

In [ ]:
items_df = count[['asin']]
print(len(items_df))

398


In [ ]:
users_df = count_users[['reviewer_id']]
print(len(users_df))

34594


In [ ]:
users_list = users_df.values
len(users_list)

34594

In [ ]:
# df1.pivot 대신 pd.pivot_table(data=df1) 사용
# as_matrix 대신 values 사용
df_clean_matrix = pd.pivot_table(data=df1, index='asin', columns='reviewer_id', values='overall').fillna(0)
df_clean_matrix = df_clean_matrix.T
R = (df_clean_matrix).values
R

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
user_ratings_mean = np.mean(R, axis=1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
R_demeaned

array([[-0.00251256, -0.00251256, -0.00251256, ..., -0.00251256,
        -0.00251256, -0.00251256],
       [-0.00753769, -0.00753769, -0.00753769, ..., -0.00753769,
        -0.00753769, -0.00753769],
       [-0.01256281, -0.01256281, -0.01256281, ..., -0.01256281,
        -0.01256281, -0.01256281],
       ...,
       [-0.00753769, -0.00753769, -0.00753769, ..., -0.00753769,
        -0.00753769, -0.00753769],
       [-0.01256281, -0.01256281, -0.01256281, ..., -0.01256281,
        -0.01256281, -0.01256281],
       [-0.01256281, -0.01256281, -0.01256281, ..., -0.01256281,
        -0.01256281, -0.01256281]])

In [ ]:
from scipy.sparse.linalg import svds

In [ ]:
U, sigma, Vt = svds(R_demeaned)

In [ ]:
sigma = np.diag(sigma)

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns=df_clean_matrix.columns)

In [ ]:
preds_df['reviewer_id'] = users_df
preds_df.set_index('reviewer_id', inplace=True)
preds_df.head()

asin,B0000302YN,B00004THXH,B00004Z2UB,B00004Z2UD,B00006B8FS,B00006IC2T,B00006IE1Q,B00006IEFO,B00006IFGJ,B0001DSIHI,...,B01FEI3FI6,B01FIY4IZU,B01FN0XGRU,B01FVAHDKI,B01G3VZ5LS,B01GGQQ8WK,B01GR8TD92,B01GSOMVRG,B01GUFGNG8,B01H64JN6O
reviewer_id,,,,,,,,,,,,,,,,,,,,,
A0193442L7IJXD7HANDM,0.002560,-0.000022,0.002546,0.002559,0.002561,0.002559,0.002559,0.002562,0.002561,0.002562,...,0.002561,0.002559,0.002557,0.002562,0.002560,0.002563,0.002564,0.002579,0.002560,0.002561
A02049035W9C30B2NBJM,-0.000896,-0.000476,0.000084,0.000015,-0.000184,0.000038,-0.000509,-0.000533,-0.002215,-0.000422,...,-0.000298,0.000001,-0.000126,-0.000342,-0.000188,-0.000529,-0.000674,-0.002994,-0.000172,-0.000259
A0228801ER3IBSLNOT5W,0.012818,-0.000307,0.012747,0.012814,0.012823,0.012814,0.012799,0.012831,0.012826,0.012829,...,0.012825,0.012815,0.012801,0.012827,0.012819,0.012833,0.012838,0.012922,0.012821,0.012824
A0345137PWD0TJ09QK1A,0.012819,-0.000364,0.012748,0.012815,0.012824,0.012815,0.012799,0.012832,0.012827,0.012830,...,0.012826,0.012816,0.012802,0.012828,0.012820,0.012834,0.012839,0.012923,0.012822,0.012825
A0429690KG9J3JO6LSSH,0.002548,0.000101,0.002536,0.002548,0.002550,0.002548,0.002545,0.002551,0.002547,0.002551,...,0.002550,0.002549,0.002546,0.002550,0.002549,0.002551,0.002552,0.002564,0.002549,0.002550


In [ ]:
preds_df.shape

(34594, 398)

In [ ]:
def recommend_it(predictions_df, itm_df, original_ratings_df, num_recommendations=10, ruserid='A0193442L7IJXD7HANDM'):
    
    # Get and sort the user's predictions
    sorted_user_predictions = predictions_df.loc[ruserid].sort_values(ascending=False)
    
    # Get the user's data and merge in the item information
    user_data = original_ratings_df[original_ratings_df.reviewer_id == ruserid]
    user_full = (user_data.merge(itm_df, how='left', left_on='asin', right_on='asin').
                sort_values(['overall'], ascending=False))
    print(f'User {ruserid} has already purchased {user_full.shape[0]} items.')
    print(f'Recommending the highest {num_recommendations} predicted items not already purchased.')
    
    # Recommend the highest predicted rating items that the user hasn't bought yet.
    recommendations = (itm_df[~itm_df['asin'].isin(user_full['asin'])].
                      merge(pd.DataFrame(sorted_user_predictions).reset_index(), how='left',
                           left_on='asin', right_on='asin').
                      rename(columns= {ruserid: 'predictions'}).
                      sort_values('predictions', ascending=False).
                      iloc[:num_recommendations, :-1])
    topk = recommendations.merge(original_ratings_df, left_index=True, right_on='asin',
                                 left_on='asin').drop_duplicates(['asin', 'title'])[['asin', 'title']]
    
    return topk

In [ ]:
recommend_it(preds_df, items_df, df1, 5)

User A0193442L7IJXD7HANDM has already purchased 1 items.
Recommending the highest 5 predicted items not already purchased.


,asin,title
450,B001QX2G9S,"Rit Dye Liquid Fabric Dye, 8-Ounce, Black"
3935,B000B7Q9KM,"Rit Dye Liquid Fabric Dye, 8-Ounce, Cocoa Brown"
1802,B00WW8KIRI,Sugru Moldable Glue - Original Formula - New C...
912,B01D3MWPU4,Singer | Heavy Duty 4411 Sewing Machine with 1...
3960,B005GXPOAM,Brother XR9500PRW Project Runway Limited Editi...


In [ ]:
# recommend for any user
recommend_it(preds_df, items_df, df1, 5, 'A1PAGHECG401K1')

User A1PAGHECG401K1 has already purchased 16 items.
Recommending the highest 5 predicted items not already purchased.


,asin,title
450,B001QX2G9S,"Rit Dye Liquid Fabric Dye, 8-Ounce, Black"
3935,B000B7Q9KM,"Rit Dye Liquid Fabric Dye, 8-Ounce, Cocoa Brown"
1802,B00WW8KIRI,Sugru Moldable Glue - Original Formula - New C...
3960,B005GXPOAM,Brother XR9500PRW Project Runway Limited Editi...
912,B01D3MWPU4,Singer | Heavy Duty 4411 Sewing Machine with 1...


In [ ]:
recommend_it(preds_df, items_df, df1, 5, 'ALYZJ7W14YS26')

User ALYZJ7W14YS26 has already purchased 17 items.
Recommending the highest 5 predicted items not already purchased.


,asin,title
7418,B004499DT2,"Snapware Yarn Tainer Storage Container, Large"
5687,B0042TJGD2,"Lion Brand Yarn 825-216 Amazing Yarn, Cobblestone"
2608,B000YZ5IY4,Environmental Technology 00876 16-Ounce Fiber-...
3585,B004W8PDWQ,"K&amp;CompanySmash Folio, Simple Orange"
3734,B00B2CCA6W,"Clover 3672 Amour Crochet Hook Set, 10 sizes"
